In [ ]:
from dabry.problem import NavigationProblem
from dabry.solver_ef import SolverEFResampling, SolverEF
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as mpl_cm
import numpy as np
import itertools

plt.rc('font', size=18)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=10)
plt.rc('mathtext', fontset='cm')
plt.rc('text', usetex=True)

In [ ]:
def my_figure(solver, ff_on=False, c_levels=np.arange(0, 1.01, 0.1), no_quiver=False, ff_sub=1, t_min=None, t_max=None,
             traj_ext=None, rain=None, times=None, use_cost_map_no_g=False):
    if t_min is None:
        t_min = solver.total_duration / 6
    if t_max is None:
        t_max = solver.total_duration
    norm = matplotlib.colors.Normalize(vmin=0, vmax=t_max, clip=True)
    c_levels = np.arange(0, t_max + 0.01, 0.1)
    cost_map = solver._cost_map if not use_cost_map_no_g else solver._cost_map_no_g
    nrows, ncols = 2, 3
    fig, axs = plt.subplots(nrows, ncols, figsize=1.1*np.array((12, 8)))
    if times is None:
        times = np.linspace(t_min, t_max, 6)
    for k, (i, j) in enumerate(itertools.product(range(nrows), range(ncols))):
        t_cur_m1 = 0 if k == 0 else times[k-1]
        t_cur = times[k]
        levels = np.hstack((c_levels[c_levels < t_cur], t_cur))
        ax = axs[i, j]
        ax.axis('equal')
        ax.set_xlim(solver.pb.bl[0], solver.pb.tr[0])
        ax.set_xbound(lower=solver.pb.bl[0], upper=solver.pb.tr[0])
        ax.set_ylim(solver.pb.bl[1], solver.pb.tr[1])
        ax.set_ybound(lower=solver.pb.bl[1], upper=solver.pb.tr[1])
        ax.set_aspect('auto')

        # Trajectories
        if rain is None:
            points_scatter = np.zeros((len(solver.trajs), 2))
            for i_traj, traj in enumerate(solver.trajs):
                sl = np.logical_and(t_cur_m1 < traj.times, traj.times < t_cur)
                ax.plot(*traj.states[sl].T, color='grey', zorder=4)
                points_scatter[i_traj, :] = traj.states[sl][-1]
            ax.scatter(*points_scatter.T, color='black', s=10, zorder=5)
            for site in solver.solution_sites:
                ax.plot(*site.traj.states[site.traj.times < t_cur].T, color='black', zorder=8)
            if solver.solution_site is not None:
                ax.plot(*solver.solution_site.traj.states[solver.solution_site.traj.times < t_cur].T, color='red', zorder=8)
                ax.scatter(*solver.solution_site.traj.states[solver.solution_site.traj.times < t_cur][-1], color='red', s=10, zorder=8)
            if traj_ext is not None:
                ax.plot(*traj_ext.states[traj_ext.times < t_cur].T, color='blue', zorder=8)
                ax.scatter(*traj_ext.states[traj_ext.times < t_cur][-1], color='blue', s=10, zorder=8)
            if not ff_on:
    
                # Cost map
                
                if k == nrows * ncols - 1:
                    c = ax.contourf(cost_map.grid_vectors[1:-1, 1:-1, 0], cost_map.grid_vectors[1:-1, 1:-1, 1],
                                cost_map.values[1:-1, 1:-1], levels=levels,
                                    cmap='jet', norm=norm, zorder=4, alpha=0.5)
                    c = ax.contour(cost_map.grid_vectors[1:-1, 1:-1, 0], cost_map.grid_vectors[1:-1, 1:-1, 1],
                                   cost_map.values[1:-1, 1:-1], levels=c_levels[c_levels < t_cur], 
                                   alpha=1, colors=((0.2, 0.2, 0.2),) if not ff_on else 'black', zorder=5)
                    ax.clabel(c, c.levels, inline=True, fontsize=15)
            else:
    
                # Flow field
                
                ff = solver.pb.model.ff
                wind_norm = matplotlib.colors.Normalize()
                wind_norm.autoscale([0, 0.85*np.linalg.norm(ff.values, axis=-1).max()])
                grid_vectors_ff = np.stack(
                    np.meshgrid(
                        np.linspace(
                            ff.bounds[-2, 0], 
                            ff.bounds[-2, 1], 
                            ff.values.shape[-3]
                        ),
                        np.linspace(
                            ff.bounds[-1, 0], 
                            ff.bounds[-1, 1], 
                            ff.values.shape[-2]
                        ), indexing= 'ij'), -1)
                grid_vectors_ff = grid_vectors_ff[::ff_sub, ::ff_sub]
                if len(ff.values.shape) == 4:
                    t_virt_ff = (t_cur - ff.bounds[0, 0]) / (ff.bounds[0, 1] - ff.bounds[0, 0])
                    it = np.floor(t_virt_ff * (ff.values.shape[0] - 1)).astype(np.int32)
                    alpha = t_virt_ff * ff.values.shape[0] - it
                    if it == ff.values.shape[0]:
                        it = ff.values.shape[0] - 2
                        alpha = 1
                    ff_frame = (1 - alpha) * ff.values[it, ::ff_sub, ::ff_sub] + alpha * ff.values[it + 1, ::ff_sub, ::ff_sub]
                else:
                    ff_frame = ff.values[::ff_sub, ::ff_sub]
                ff_norms = np.linalg.norm(ff_frame, axis=-1)
                cmesh = ax.pcolormesh(grid_vectors_ff[..., 0], grid_vectors_ff[..., 1], 
                              ff_norms, zorder=2,
                              shading='gouraud', cmap='turbo', norm=wind_norm)
                rect = patches.Rectangle(solver.pb.bl, solver.pb.tr[0] - solver.pb.bl[0], solver.pb.tr[1] - solver.pb.bl[1],
                                    alpha=0.3, color='white', zorder=3)
                ax.add_patch(rect)
                if not no_quiver:
                    ax.quiver(grid_vectors_ff[..., 0], grid_vectors_ff[..., 1], ff_frame[..., 0], ff_frame[..., 1], zorder=4)
            
        # Obstacle
        
        obs = solver.pb.obstacles[0]
        if obs.__class__.__name__ != 'FrameObs':
            grid_vectors_obs = np.stack(
                np.meshgrid(
                    np.linspace(
                        obs.bounds[-2, 0], 
                        obs.bounds[-2, 1], 
                        obs.values.shape[-2]
                    ),
                    np.linspace(
                        obs.bounds[-1, 0], 
                        obs.bounds[-1, 1], 
                        obs.values.shape[-1]
                    ), indexing= 'ij'), -1)
            t_virt_obs = (t_cur - obs.bounds[0, 0]) / (obs.bounds[0, 1] - obs.bounds[0, 0])
            it = np.floor(t_virt_obs * (obs.values.shape[0] - 1)).astype(np.int32)
            alpha = t_virt_obs * obs.values.shape[0] - it
            if it == obs.values.shape[0]:
                it = obs.values.shape[0] - 2
                alpha = 1
            obs_frame = (1 - alpha) * obs.values[it] + alpha * obs.values[it + 1]
            ax.contourf(grid_vectors_obs[..., 0], grid_vectors_obs[..., 1], 
                        obs_frame, levels=[-100, 0],
                        colors='purple', extend='min', alpha=0.5, zorder=4)
            ax.contour(grid_vectors_obs[..., 0], grid_vectors_obs[..., 1], obs_frame, levels=[0], colors='purple', zorder=4)

        # Rain

        if rain is not None:
            ff = solver.pb.model.ff
            grid_vectors_ff = np.stack(
                np.meshgrid(
                    np.linspace(
                        ff.bounds[-2, 0], 
                        ff.bounds[-2, 1], 
                        ff.values.shape[-3]
                    ),
                    np.linspace(
                        ff.bounds[-1, 0], 
                        ff.bounds[-1, 1], 
                        ff.values.shape[-2]
                    ), indexing= 'ij'), -1)
            grid_vectors_ff = grid_vectors_ff[::ff_sub, ::ff_sub]
            if len(ff.values.shape) == 4:
                t_virt_ff = (t_cur - ff.bounds[0, 0]) / (ff.bounds[0, 1] - ff.bounds[0, 0])
                it = np.floor(t_virt_ff * (ff.values.shape[0] - 1)).astype(np.int32)
                alpha = t_virt_ff * ff.values.shape[0] - it
                if it == ff.values.shape[0]:
                    it = ff.values.shape[0] - 2
                    alpha = 1
                ff_frame = (1 - alpha) * ff.values[it, ::ff_sub, ::ff_sub] + alpha * ff.values[it + 1, ::ff_sub, ::ff_sub]
            else:
                ff_frame = ff.values[::ff_sub, ::ff_sub]
            ff_norms = np.linalg.norm(ff_frame, axis=-1)
            ax.pcolormesh(grid_vectors_ff[..., 0], grid_vectors_ff[..., 1], 
                          ff_norms, zorder=2,
                          shading='gouraud', cmap='turbo')
            rect = patches.Rectangle(solver.pb.bl, solver.pb.tr[0] - solver.pb.bl[0], solver.pb.tr[1] - solver.pb.bl[1],
                                alpha=0.3, color='white', zorder=3)
            ax.add_patch(rect)
            if not no_quiver:
                ax.quiver(grid_vectors_ff[..., 0], grid_vectors_ff[..., 1], ff_frame[..., 0], ff_frame[..., 1], zorder=4)
            
        
        # Origin destination
        
        ax.scatter(*solver.pb.x_init, color='black', edgecolor='white', s=100, zorder=10)
        ax.scatter(*solver.pb.x_target, color='black', edgecolor='white', s=200, marker='*', zorder=10)
        circ = patches.Circle(solver.pb.x_target, solver.target_radius, facecolor='none', edgecolor='black', linewidth=1, zorder=8)
        ax.add_patch(circ)
        
        ax.set_title(rf'$t={t_cur:.3g}$')
        ax.grid(True)
    return fig, axs

# Custom problems

## Moving Disk

In [ ]:
from dabry.flowfield import ZeroFF
from dabry.obstacle import DiscreteObs

v_a = 1

x_init = np.array((0.15, 0.5))
x_target = np.array((0.85, 0.5))
bounds = np.array(((0, 1), (0, 1), (0, 1)))

ff = ZeroFF()

shape = (25, 51, 51)
grid_vectors_st = np.stack(np.meshgrid(
    *(np.linspace(bounds[i, 0], bounds[i, 1], shape[i]) for i in range(3)),
    indexing='ij'), -1)

center = np.stack((np.linspace(0.5, 0.5, shape[0]), np.linspace(0.7, 0.3, shape[0])), -1)
radius = 0.1
diffs = (grid_vectors_st[..., 1:].transpose((1, 2, 0, 3)) - center).transpose((2, 0, 1, 3))
values = np.linalg.norm(diffs, axis=-1) - radius
values = np.linalg.norm(diffs, axis=-1) - radius
obs = DiscreteObs(values, bounds)

a = obs.event(0.5, np.array((0.5, 0.5, 0.5)))

pb = NavigationProblem(ff, x_init, x_target, v_a, bl=bounds[1:].T[0], tr=bounds[1:].T[1],
                       name='Moving circle', obstacles=[obs])

In [ ]:
solver = SolverEFResampling(pb, free_max_step=False, tangency_tol=np.pi/2)
solver.solve()

In [ ]:
fig, axs = my_figure(solver)
for ax in axs.flatten():
    ax.set_xticks(np.arange(0, 1.1, 0.2))
#fig.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_moving/disk.pdf')
plt.show()

In [ ]:
grid_vectors_obs = np.stack(
    np.meshgrid(
        np.linspace(
            obs.bounds[-2, 0], 
            obs.bounds[-2, 1], 
            obs.values.shape[-2]
        ),
        np.linspace(
            obs.bounds[-1, 0], 
            obs.bounds[-1, 1], 
            obs.values.shape[-1]
        ), indexing= 'ij'), -1)

In [ ]:
dvalues = np.zeros(grid_vectors_obs.shape[:-1])
for i in range(grid_vectors_obs.shape[0]):
    for j in range(grid_vectors_obs.shape[1]):
        dvalues[i, j] = obs.d_value_dt(0.5, grid_vectors_obs[i, j])
plt.contourf(grid_vectors_obs[..., 0], grid_vectors_obs[..., 1], dvalues)

In [ ]:
obs.d_value_dt(0, np.array((0.5, 0.5))) / np.linalg.norm(obs.d_value(0, np.array((0.5, 0.5))))

## Moving Barrier

In [ ]:
from dabry.flowfield import ZeroFF
from dabry.obstacle import DiscreteObs

v_a = 1

x_init = np.array((0.15, 0.5))
x_target = np.array((0.85, 0.5))
bounds = np.array(((0, 1), (0, 1), (0, 1)))

ff = ZeroFF()

In [ ]:
import shapely

dist = 0.025

bar = np.array((np.linspace(0, 0, 50), np.linspace(-0.2, 0.2, 50))).T

shape = (25, 51, 51)
grid_vectors_st = np.stack(np.meshgrid(
    *(np.linspace(bounds[i, 0], bounds[i, 1], shape[i]) for i in range(3)), 
    indexing='ij'), -1)

values = np.zeros(shape)
for k in range(shape[0]):
    for i in range(shape[1]):
        for j in range(shape[2]):
            center = np.array((0.5, 0.7)) - np.array((0, 0.4)) * k / (shape[0] - 1)
            points = bar + center
            line1 = shapely.LineString(points)
            values[k, i, j] = line1.distance(shapely.Point(grid_vectors_st[0, i, j, 1:])) - dist

In [ ]:
it = 24
plt.contour(grid_vectors_st[it, ..., 1], grid_vectors_st[it, ..., 2], values[it], levels=[0])

In [ ]:
obs = DiscreteObs(values, bounds)

a = obs.event(0.5, np.array((0.5, 0.5, 0.5)))

pb = NavigationProblem(ff, x_init, x_target, v_a, bl=bounds[1:].T[0], tr=bounds[1:].T[1],
                       name='Moving bar', obstacles=[obs])

In [ ]:
solver = SolverEFResampling(pb, free_max_step=False, tangency_tol=np.pi/2)
solver.solve()

In [ ]:
fig, ax = my_figure(solver)
for ax in axs.flatten():
    ax.set_xticks(np.arange(0, 1.1, 0.2))
fig.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_moving/bar.pdf')
plt.show()

## Linear, moving bar

In [ ]:
from dabry.flowfield import StateLinearFF
from dabry.obstacle import DiscreteObs

v_a = 1

x_init = np.array((0.15, 0.5))
x_target = np.array((0.85, 0.5))
bounds = np.array(((0, 1), (0, 1), (0, 1)))

ff_a = StateLinearFF(np.array(((0, 1), (0, 0))), np.array((0, 0.5)), np.array((0, 0)))

In [ ]:
import shapely

dist = 0.025

bar = np.array((np.linspace(0, 0, 50), np.linspace(-0.2, 0.2, 50))).T

shape = (25, 51, 51)
grid_vectors_st = np.stack(np.meshgrid(
    *(np.linspace(bounds[i, 0], bounds[i, 1], shape[i]) for i in range(3)), 
    indexing='ij'), -1)

values = np.zeros(shape)
for k in range(shape[0]):
    for i in range(shape[1]):
        for j in range(shape[2]):
            center = np.array((0.7, 0.5)) - np.array((0.6, 0)) * k / (shape[0] - 1)
            points = bar + center
            line1 = shapely.LineString(points)
            values[k, i, j] = line1.distance(shapely.Point(grid_vectors_st[0, i, j, 1:])) - dist

In [ ]:
it = 15
plt.contour(grid_vectors_st[it, ..., 1], grid_vectors_st[it, ..., 2], values[it], levels=[0])

In [ ]:
from dabry.flowfield import DiscreteFF
ff = DiscreteFF.from_ff(ff_a, bounds[1:])

In [ ]:
obs = DiscreteObs(values, bounds)

pb = NavigationProblem(ff, x_init, x_target, v_a, bl=bounds[1:].T[0], tr=bounds[1:].T[1],
                       name='Moving bar', obstacles=[obs])

In [ ]:
solver = SolverEFResampling(pb, free_max_step=False)
solver.solve()

In [ ]:
fig, axs = my_figure(solver, t_min=0.3, t_max=0.74, ff_on=True, ff_sub=3)
for ax in axs.flatten():
    ax.set_xticks(np.arange(0, 1.1, 0.2))
fig.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_moving/linear_bar.pdf')
plt.show()

## Road crossing

In [ ]:
from dabry.flowfield import StateLinearFF, ZeroFF
from dabry.obstacle import DiscreteObs

v_a = 1

x_init = np.array((0.15, 0.5))
x_target = np.array((0.85, 0.5))
bounds = np.array(((0, 1), (0, 1), (0, 1)))

ff = ZeroFF() # StateLinearFF(np.array(((0, 1), (0, 0))), np.array((0, 0.5)), np.array((0, 0)))

In [ ]:
import shapely

n_bar = 50

class Vehicle:
    def __init__(self, width, length):
        self.width = width
        self.length = length
        self.bar = np.array((np.linspace(0, 0, n_bar), np.linspace(-self.length/2, self.length/2, n_bar))).T

class MovingVehicle:
    def __init__(self, vehicle, pos_init, velocity):
        self.vehicle = vehicle
        self.pos_init = pos_init 
        self.velocity = velocity

    def bar(self, t):
        return self.vehicle.bar + self.pos_init + self.velocity * t

car = Vehicle(0.05, 0.1)
truck = Vehicle(0.05, 0.2)

mvs = [
    MovingVehicle(car, np.array((0.5, 0.5)), np.array((0, -0.5))),
    MovingVehicle(car, np.array((0.5, 0.8)), np.array((0, -0.5))),
    MovingVehicle(truck, np.array((0.7, 0.4)), np.array((0, 0.3))),
]

shape = (25, 101, 101)
grid_vectors_st = np.stack(np.meshgrid(
    *(np.linspace(bounds[i, 0], bounds[i, 1], shape[i]) for i in range(3)), 
    indexing='ij'), -1)

times = np.linspace(0, 1, shape[0])
values = np.inf * np.ones(shape)
for k in range(shape[0]):
    for i in range(shape[1]):
        for j in range(shape[2]):
            for mv in mvs:
                line = shapely.LineString(mv.bar(times[k]))
                values[k, i, j] = min(line.distance(shapely.Point(grid_vectors_st[0, i, j, 1:])) - mv.vehicle.width, values[k, i, j])

In [ ]:
it = 20
plt.contour(grid_vectors_st[it, ..., 1], grid_vectors_st[it, ..., 2], values[it], levels=[0])

In [ ]:
obs = DiscreteObs(values, bounds)

pb = NavigationProblem(ff, x_init, x_target, v_a, bl=bounds[1:].T[0], tr=bounds[1:].T[1],
                       name='Road crossing', obstacles=[obs])

In [ ]:
from dabry.solver_ef import SolverEFResampling
solver = SolverEFResampling(pb)
solver.solve()

In [ ]:
fig, ax = my_figure(solver, t_min=0.28, t_max=0.72)
for ax in axs.flatten():
    ax.set_xticks(np.arange(0, 1.1, 0.2))
fig.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_moving/road_crossing.pdf')
plt.show()

## Storm Dakar Natal

In [ ]:
import scipy.io
from dabry.flowfield import DiscreteFF
from dabry.misc import Coords
x_init = np.array([0.8883191 , 1.05880435])
x_target = np.array((0, 0))
srf = 1.39725
vel_haz = scipy.io.loadmat('../../data/Dakar_Natal_2021Sep28_storm/scaled/velocity_hazard.mat')
ff_values = vel_haz['ff_values']
bounds = vel_haz['bounds']
ff = DiscreteFF(ff_values, bounds, Coords.CARTESIAN)

In [ ]:
hazard_values = vel_haz['hazard_values']
grid_vectors_st = np.stack(np.meshgrid(
    *(np.linspace(bounds[i, 0], bounds[i, 1], ff_values.shape[i]) for i in range(3)), 
    indexing='ij'), -1)
centers = np.zeros((ff_values.shape[0], 2))
for it in range(ff_values.shape[0]):
    idx = np.unravel_index(hazard_values[it].argmin(), hazard_values[it].shape)
    centers[it, :] = grid_vectors_st[it, *idx][1:]

In [ ]:
from scipy.signal import convolve
n_conv = 10
centers_conv = np.stack((convolve(centers[:, 0], np.ones(n_conv)/n_conv, mode='same'),
                         convolve(centers[:, 1], np.ones(n_conv)/n_conv, mode='same')), -1)

In [ ]:
for i in range(5):
    centers_conv[i, :] = centers_conv[5]
for i in range(4):
    centers_conv[-1 - i, :] = centers_conv[-5]

In [ ]:
plt.scatter(*centers.T)
plt.scatter(*centers_conv.T)

In [ ]:
dt = (bounds[0, 1] - bounds[0, 0]) / (ff_values.shape[0] - 1)
print(dt)

In [ ]:
np.linalg.norm(centers_conv[1:] - centers_conv[:-1], axis=-1)/dt

In [ ]:
obs_values = np.zeros(ff_values.shape[:-1])
for it in range(obs_values.shape[0]):
    radius = 0.2
    obs_values[it, :] = np.linalg.norm(grid_vectors_st[it, ..., 1:] - centers_conv[it], axis=-1) - radius

In [ ]:
# obs_values = np.zeros(ff_values.shape[:-1])
# for it in range(obs_values.shape[0]):
#     obs_values[it, :] = hazard_values[it] + 0.5
# n_conv = 4
# for it in range(0, ff_values.shape[0], 4):
#     plt.contour(convolve(obs_values[it], np.ones((n_conv, n_conv)) / n_conv ** 2, mode='same'), levels=[0], 
#                 colors=(matplotlib.colormaps.get_cmap('jet')(it/(ff_values.shape[0] - 1)),))

### No obstacle

In [ ]:
pb = NavigationProblem(ff, x_init, x_target, srf, bl=bounds[1:].T[0], tr=bounds[1:].T[1],
                       name='Storm Dakar Natal no obs')

In [ ]:
solver_no_obs = SolverEFResampling(pb, total_duration=1.1, max_depth=25)
solver_no_obs.solve()

In [ ]:
fig, axs = my_figure(solver_no_obs, ff_on=True, ff_sub=3)
for ax in axs.flatten():
    ax.set_xticks(np.arange(0, 1.1, 0.2))
#fig.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_moving/storm_dakar_natal_no_obs.png', dpi=300)
plt.show()

### With obstacle

In [ ]:
from dabry.obstacle import DiscreteObs
obs = DiscreteObs(obs_values, bounds)

In [ ]:
pb = NavigationProblem(ff, x_init, x_target, srf, bl=bounds[1:].T[0], tr=bounds[1:].T[1],
                       name='Storm Dakar Natal', obstacles=[obs])

In [ ]:
ff.values.shape

In [ ]:
solver = SolverEFResampling(pb, total_duration=1.1, max_depth=25)
solver.solve()

In [ ]:
fig, axs = my_figure(solver, ff_on=True, ff_sub=3, traj_ext=solver_no_obs.solution_site.traj)
for ax in axs.flatten():
    ax.set_xticks(np.arange(-0.2, 1.1, 0.2))
    ax.set_ybound(-0.23, 1.195)
    ax.set_yticks(np.arange(-0.2, 1.21, 0.2))
fig.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_moving/storm_dakar_natal.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
wind_norm = matplotlib.colors.Normalize()
wind_norm.autoscale([0, 0.85*np.linalg.norm(solver.pb.model.ff.values, axis=-1).max()])
plt.colorbar(mpl_cm.ScalarMappable(cmap='turbo', norm=wind_norm),
                     label="Wind norm [no dim.]", location='right', fraction=2*.032, pad=0.04) 
plt.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_moving/storm_dakar_natal_cmap.png', dpi=300)

In [ ]:
solver.pb.time_radial()

In [ ]:
solver.pb.x_target

# Fast disk

In [ ]:
from dabry.flowfield import ZeroFF
from dabry.obstacle import DiscreteObs

v_a = 1

x_init = np.array((0.15, 0.5))
x_target = np.array((0.85, 0.5))
bounds = np.array(((0, 1), (0, 1), (0, 1)))

ff = ZeroFF()

shape = (51, 51, 51)
grid_vectors_st = np.stack(np.meshgrid(
    *(np.linspace(bounds[i, 0], bounds[i, 1], shape[i]) for i in range(3)),
    indexing='ij'), -1)

center = np.stack((np.linspace(0.5, 0.5, shape[0]), np.linspace(1.2, -1.2, shape[0])), -1)
radius = 0.1
diffs = (grid_vectors_st[..., 1:].transpose((1, 2, 0, 3)) - center).transpose((2, 0, 1, 3))
values = np.linalg.norm(diffs, axis=-1) - radius
values = np.linalg.norm(diffs, axis=-1) - radius
obs = DiscreteObs(values, bounds)

a = obs.event(0.5, np.array((0.5, 0.5, 0.5)))

pb = NavigationProblem(ff, x_init, x_target, v_a, bl=bounds[1:].T[0], tr=bounds[1:].T[1],
                       name='Fast disk', obstacles=[obs])

In [ ]:
solver = SolverEFResampling(pb, free_max_step=False, tangency_tol=np.pi/2, max_depth=25)
solver.solve()

In [ ]:
solver.get_cost_map_no_guarantee()

In [ ]:
fig, axs = my_figure(solver, times=[0.24, 0.28, 0.32, 0.36, 0.54, 0.72], use_cost_map_no_g=True)
for ax in axs.flatten():
    ax.set_xticks(np.arange(0, 1.1, 0.2))
fig.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/obstacle_moving/fast_disk.pdf', bbox_inches='tight')
plt.show()